In [ ]:
import scrapy
import logging
import json
import codecs
import pandas as pd
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
import requests
from bson import json_util
from github import Github
import xlsxwriter
import numpy as np
from itertools import cycle
import matplotlib.pyplot as plt

In [ ]:
fdroid_df = pd.read_json('obfuscated.json')
fdroid_df.head(1)

In [ ]:
fdroid_df.shape

In [ ]:
fdroid_df = fdroid_df.drop(columns = ['depth','download_timeout','download_slot','download_latency','redirect_times','redirect_ttl','redirect_urls','redirect_reasons'])

In [ ]:
fdroid_df.shape

In [ ]:
fdroid_df.head(10)

In [ ]:
fdroid_df[fdroid_df["source_code_repo"].isna()]

In [ ]:
droid_df_no_none_source_issue = fdroid_df[  (~fdroid_df["source_code_repo"].isna()) ]

In [ ]:
droid_df_no_none_source_issue.shape

In [ ]:
fdroid_df_github = droid_df_no_none_source_issue[(droid_df_no_none_source_issue["source_code_repo"].str.startswith('https://github.com'))]

In [ ]:
fdroid_df_github

In [ ]:
fdroid_df_github.shape

In [ ]:
fdroid_df_github["creation_date"] = None
fdroid_df_github["size_reported_github"] = None
fdroid_df_github["main_language"] = None
fdroid_df_github["repo_name_on_github"] = None

In [ ]:
fdroid_df_github.reset_index(drop=True, inplace=True)

In [ ]:
def getToken(token_number):
    if str(token_number) == '1':
        return 'obfuscated'
    if str(token_number) == '2':
        return 'obfuscated'
    if str(token_number) == '3':
        return 'obfuscated'
    if str(token_number) == '4':
        return 'obfuscatedv'
    if str(token_number) == '5':
        return 'obfuscated'
    if str(token_number) == '6':
        return 'obfuscated'
    if str(token_number) == '7':
        return 'obfuscated'
    if str(token_number) == '8':
        return 'obfuscated'
    if str(token_number) == '9':
        return 'obfuscated'
    if str(token_number) == '10':
        return 'obfuscated'
    if str(token_number) == '11':
        return 'obfuscated'
    if str(token_number) == '12':
        return 'obfuscated'

In [ ]:
## Cuando el desarrollador marca el repo como archivado, no hay informacion disponible para el API
## Si el nombre del repo termina con / al meterlo a la URL del API, no lo encuentra (e.g., https://github.com/ElijaxApps-org/android-xmrig-miner/)
## Cuidado con el /tree envia a una rama especifica... y no encuentra info al ser llamada a la API

def process(dataframe, first_token, second_token, third_token):
    dft = dataframe
    tkn_array = [getToken(first_token), getToken(second_token), getToken(third_token)]
    alternator = cycle((0,1,2))
    active_tkn = tkn_array[next(alternator)]
    headers = {'Authorization': 'token ' + str(active_tkn)}
    
    for i, row in dft.iterrows():
        if row['creation_date'] == None:
            repo = row['source_code_repo'].replace("https://github.com/", "").split("tree/", 1)[0]
            repo = repo[:-1] if repo.endswith('/') else repo
            print(repo)
            try:
                response = requests.get("https://api.github.com/repos/"+repo, headers=headers)
                
                if '"message": "Not Found"' not in response:
                    data = json.dumps(response.json())
                    loadAsJson = json.loads(data)

                    year = loadAsJson["created_at"]
                    size = loadAsJson["size"]
                    language = loadAsJson["language"]
                    name = loadAsJson["name"]

                    dft.at[i, 'creation_date'] = year
                    dft.at[i, 'size_reported_github'] = size
                    dft.at[i, 'main_language'] = str(language)
                    dft.at[i, 'repo_name_on_github'] = name

                    print("--------------------------------------------------------------------------------------------------")
                    print(str(i), " Estoy evaluando: ", name +" "+ year +" "+ str(size) +" "+ language)
                    print("--------------------------------------------------------------------------------------------------")
                else:
                    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                    print (str(i), " NOT FOUND", row['source_code_repo'])
                    print (" Repo name: ", row['app_name'])
                    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                    dft.at[i, 'creation_date'] = 'Not Found'
                    dft.at[i, 'size_reported_github'] = 'Not Found'
                    dft.at[i, 'main_language'] = 'Not Found'
                    dft.at[i, 'repo_name_on_github'] = 'Not Found'
            except Exception as e:
                print("##################################################################################################")
                print (str(i), " Estoy evaluando: ", row['source_code_repo'])
                print (" Repo name: ", row['app_name'])
                print ("Causa la excepcion: ", e)
                print("##################################################################################################")
                dft.at[i, 'creation_date'] = None
                dft.at[i, 'size_reported_github'] = None
                dft.at[i, 'main_language'] = None
                dft.at[i, 'repo_name_on_github'] = None
    return dft

In [ ]:
fdroid_df_github = process(fdroid_df_github, 4,5,6)

In [ ]:
fdroid_df_github

In [ ]:
count_nan = len(fdroid_df_github) - fdroid_df_github.count()
print(str(count_nan))

In [ ]:
droid_df_clean = fdroid_df_github[  (~fdroid_df_github["main_language"].isna()) ]

In [ ]:
droid_df_clean.shape

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('obfuscated.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
droid_df_clean.to_excel(writer, sheet_name='obfuscated')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
plt.clf()
droid_df_clean.groupby('main_language').size().sort_values(ascending=False).plot(kind='bar',figsize=(20,10))
plt.show()

In [ ]:
droid_df_clean.groupby('main_language').size().sort_values(ascending=False)

In [ ]:
res = g.apply(lambda x: x.order(ascending=False).head(3))